In [ ]:
from toml import load as toml_load
import matplotlib.pyplot as plt
import numpy as np

from utils import get_df, collate
from my_config import MyConfig
from embedding_model.config import Config

In [ ]:
config = Config(**toml_load("config.toml"))
my_config = MyConfig(**toml_load("my_config.toml"))

labels = list(my_config.dict().keys())

model_name = "DTNN"
yob_cutoff = 2020
followup_cutoff = 0

In [ ]:
fig, axes = plt.subplots(1, len(labels), figsize=(16, 4))
for index, label in enumerate(labels):
    print(f"Processing {label}...")
    myconfig_phenotype = getattr(my_config, label)
    config_phenotype = getattr(config.phenotypes, label)
    num_bins = len(config_phenotype.bin_boundaries) - 1
    model = myconfig_phenotype.load_model(model_name, yob_cutoff, followup_cutoff, num_bins)

    df = get_df(
        myconfig_phenotype.filepaths.test1422,
        yob_cutoff,
        followup_cutoff,
        config_phenotype.age_cutoff,
    )

    _, _, test_predictions = collate(model, df, myconfig_phenotype.vocab, config.model.seq_threshold, label)
    
    pred_mean = np.mean(test_predictions, axis=0)
    pred_lower = np.percentile(test_predictions, 2.5, axis=0)
    pred_upper = np.percentile(test_predictions, 97.5, axis=0)

    axes[index].bar(
        np.arange(num_bins + 1),
        np.mean(test_predictions, axis=0),
        yerr=np.stack([
            pred_mean - pred_lower,
            pred_upper - pred_mean
        ]),
        color=myconfig_phenotype.color
    )

    axes[index].set_xlabel('Bin')
    axes[index].set_ylabel('Predicted Event Probability')
    axes[index].set_title(myconfig_phenotype.name)
    axes[index].xaxis.set_ticks(np.arange(0, num_bins + 1, 1))

plt.tight_layout()
plt.show()